In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import pandas as pd
import numpy as np
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [4]:
# Data Load
def Data_Load():
    global df
    df = pd.read_csv('D:/Denoising/Autoencoder/LSTM_SAE/Samsung.txt', sep = ',')
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df[['CLOSE']])
    df["Close"] = scaled
    return df

In [5]:
def Slice_Window(size):
    global window_list, x_train, x_test, y_train, y_test
    window_list = np.zeros((len(df)-size+1,size,1))
    count = 0
    for i in range(len(df)-size+1):
        end_idx = i+size
        dff = df[i:end_idx]
        dff.reset_index(inplace = True)
        if len(dff)>=size:
            for j in range(size):
                window_list[count][j][0] = dff["Close"][j]            
            count += 1
    
    # Split Train and Test Set
    x_train, x_test, y_train, y_test = train_test_split(window_list, window_list, test_size=0.2, random_state=42)

In [11]:
def Build_Model():
    global model
    model = keras.Sequential()

    timestamp = window_list.shape[1]
    features = window_list.shape[2]

    # Encoder LSTM cell1
    model.add(keras.layers.LSTM(units=20, input_shape=(timestamp, features), return_sequences = True))
    model.add(keras.layers.Dropout(rate=0.2))

    # Encoder LSTM cell2
    model.add(keras.layers.LSTM(units=15, return_sequences=True))
    model.add(keras.layers.Dropout(rate=0.2))

    # Encoder LSTM cell3
    model.add(keras.layers.LSTM(units=10, return_sequences=False))
    model.add(keras.layers.Dropout(rate=0.2))

    model.add(keras.layers.RepeatVector(timestamp))

    # Decoder LSTM cell1
    model.add(keras.layers.LSTM(units=10, return_sequences=True))
    model.add(keras.layers.Dropout(rate=0.2))

    # Decoder LSTM cell2
    model.add(keras.layers.LSTM(units=15, return_sequences=True))
    model.add(keras.layers.Dropout(rate=0.2))

    # Decoder LSTM cell3
    model.add(keras.layers.LSTM(units=20, return_sequences=True))
    model.add(keras.layers.Dropout(rate=0.2))

    model.add(keras.layers.TimeDistributed(keras.layers.Dense(features)))
    model.compile(loss='mse', optimizer = Adam(learning_rate = 0.0001))

    return model.summary()

In [12]:
def Output():
    global mean_list, mean_list2
    # Sum all window size units
    mean_list = [0 for i in range(len(df))]
    for i in range(len(window_list)):
        for j in range(20):
            mean_list[i+j] += pred[i][j]

    # Calculate average
    mean_list2= []
    count = 19
    for i in range(len(mean_list)):
        if i < 20:
            mean_list2.append(mean_list[i] / (i+1))
        elif i >=20 and (i <= len(mean_list)-20):
            mean_list2.append(mean_list[i] / 20)
        else:
            mean_list2.append(mean_list[i] / count)
            count -= 1
    

In [17]:
# Check Output MAE
def Check_MAE(idx):
    global total_mae, total_mae2
    total_mae = 0
    total_mae2 = 0
    stock_price = list(df["Close"])
    for i in range(len(window_list)):
        mae = np.abs(stock_price[i] - window_list[idx][i])
        total_mae += mae
    total_mae2 = total_mae / len(window_list[idx])*100
    return total_mae2
